# Using D3 to Visualize and Interact with Distribution Grid Simulation Data

#### Monte Lunacek

<img src="files/img/working_with_d3.png" style="width:500px">

## Outline


####How I use `d3.js` at NREL.

- `Python` + `d3.js`
- Motivating examples (maybe)
   
[Construction](http://mlunacek.org/15_construction/index.html)

[Markets and EVs](http://mlunacek.org/19_real_time/index.html)
   
[HEMS and solar](http://mlunacek.org/22_hems_solar_pen/index.html)

[Voltage and Load](http://mlunacek.org/01_voltage/index.html)

####What are the key challenges when adding interaction?
* `javascript` as a data structure manipulation tool...
* Selecting objects from objects.
   
####Learn by Example

[1. graph](01_graph.html)

[2. line](02_lines.html)

[3. graph + line](03_together.html)

[4. voronoi graph](04_voronoi_graph.html)

[5. voronoi interaction](05_voronoi_together.html)

[6. final](06_lines.html)

####Links

[http:http://mlunacek.org/21_meetup_d3_2015/](http:http://mlunacek.org/21_meetup_d3_2015/)

[https://github.com/mlunacek/meetup_d3_2015](https://github.com/mlunacek/meetup_d3_2015)
   
[http://learnjsdata.com/](http://learnjsdata.com/)

[IPython notebook](https://store.continuum.io/cshop/anaconda/)

[iJavascript](https://www.npmjs.com/package/ijavascript)




#Data 

In [1]:
var fs = require("fs");
var d3 = require("d3");
var _  = require("lodash");

undefined

## 1. Network Graph 

This replaces the following code in the browser:

        d3.json("data/example_nodes.json", function(nodes) {
            d3.json("data/example_edges.json", function(edges) {
        
                // do something with nodes and edges
              
            });
        });

In [2]:

var nodes, edges;

fs.readFile("data/example_nodes.json", "utf8", function(error, node_data) {
    fs.readFile("data/example_edges.json", "utf8", function(error, edge_data) {
        nodes = JSON.parse(node_data);
        edges = JSON.parse(edge_data);
    });
});

undefined

What does this look like?

In [3]:
nodes.slice(0,5)

[ { y: -169.37, x: 90.19, name: 'B11' },
  { y: 179.46, x: -38.979, name: 'B0' },
  { y: 80.9, x: 85.935, name: 'B1' },
  { y: 229.13, x: -99.801, name: 'B2' },
  { y: -60.999, x: -85.874, name: 'B3' } ]

In [4]:
edges.slice(0,5)

[ { source: 'B0', target: 'B9' },
  { source: 'B0', target: 'B2' },
  { source: 'B4', target: 'B5' },
  { source: 'B5', target: 'B1' },
  { source: 'B5', target: 'B12' } ]

### Scales

We could use `d3.max()` and `d3.min()`.

In [5]:
var min_x = d3.min(nodes, function(d){ return d['x'] });
var max_x = d3.max(nodes, function(d){ return d['x'] });

var x_scale = d3.scale.linear()
                .domain([min_x, max_x])
                .range([0, 100]);

x_scale(75.0)

64.61642534221002

### lodash

In [6]:
nodes.slice(0,5)

[ { y: -169.37, x: 90.19, name: 'B11' },
  { y: 179.46, x: -38.979, name: 'B0' },
  { y: 80.9, x: 85.935, name: 'B1' },
  { y: 229.13, x: -99.801, name: 'B2' },
  { y: -60.999, x: -85.874, name: 'B3' } ]

In [7]:
var xvals = _.pluck(nodes, "x")
var yvals = _.pluck(nodes, "y")

yvals.slice(0, 5)

[ -169.37, 179.46, 80.9, 229.13, -60.999 ]

And then use `d3.extent()`.

In [8]:
d3.extent(yvals)

[ -225.11, 229.13 ]

In [9]:
var x_scale = d3.scale.linear()
                .domain(d3.extent(xvals))
                .range([0, 100]);

var y_scale = d3.scale.linear()
                .domain(d3.extent(yvals))
                .range([100, 0]);


undefined

### Add the `x,y` values to the edges dataset.

The `javascript` `Array.prototype.forEach` method executes a provided function once per array element.

We can use this to convert our `x` and `y` values.  Only call this **once** if you do it this way.

In [10]:
nodes.forEach(function(d) {
        d.x = x_scale(d.x);
        d.y = y_scale(d.y); 
      });

nodes[0]

{ y: 87.7289538569919, x: 70.23151622240047, name: 'B11' }

Let's create a fast *lookup* for these values.

In [11]:
node_map = {}
nodes.forEach(function(d) { node_map[d.name] = d; });

node_map

{ B11: { y: 87.7289538569919, x: 70.23151622240047, name: 'B11' },
  B0: { y: 10.9347481507573, x: 22.48328225904828, name: 'B0' },
  B1: { y: 32.632529059528004, x: 68.6586253932227, name: 'B1' },
  B2: { y: 0, x: 0, name: 'B2' },
  B3: { y: 63.87130151461782, x: 5.148214001870468, name: 'B3' },
  B10: { y: 100, x: 89.05815075354593, name: 'B10' },
  B6: { y: 77.36218738992604, x: 46.36201995408859, name: 'B6' },
  B7: { y: 42.31771750616414, x: 42.2407132902806, name: 'B7' },
  B8: { y: 91.13464247974639, x: 26.55986041749069, name: 'B8' },
  B13: { y: 60.25449101796407, x: 35.45358770668451, name: 'B13' },
  B4: { y: 6.879623106727717, x: 100, name: 'B4' },
  B9: { y: 29.013957379358935, x: 20.507834881580358, name: 'B9' },
  B12: { y: 3.0644593166607836, x: 50.25894477693045, name: 'B12' },
  B5: { y: 14.70368087354702, x: 74.08334288280761, name: 'B5' } }

And then add these values to the `edges` data with the `Array.prototype.forEach`.

In [12]:
edges.forEach(function(d) {
        d.x1 = node_map[d.source].x;
        d.y1 = node_map[d.source].y;
        d.x2 = node_map[d.target].x;
        d.y2 = node_map[d.target].y;
    });

edges.slice(0,2)

[ { source: 'B0',
    target: 'B9',
    x1: 22.48328225904828,
    y1: 10.9347481507573,
    x2: 20.507834881580358,
    y2: 29.013957379358935 },
  { source: 'B0',
    target: 'B2',
    x1: 22.48328225904828,
    y1: 10.9347481507573,
    x2: 0,
    y2: 0 } ]

## 2. Load Lines 

Load the `csv` data.

<img src="files/img/csv_table.png">

        d3.csv("data/example_load.csv", function(load) {

            //do something with load

        });

In [14]:
var load;

fs.readFile("data/example_load.csv", "utf8", function(error, load_data) {
    load = d3.csv.parse(load_data);
});

undefined

When we parse this with `d3`, we get the following `json` structure.

In [15]:
load.slice(0,2)

[ { index: '2020-04-16 00:00:00',
    B10: '0.857212669594',
    B3: '7.67566995284',
    B4: '1.60806522167',
    B5: '7.68251588423',
    B6: '0.514955844603',
    B7: '0.642797357164',
    B0: '3.21335934831',
    B1: '1.54462066449',
    B2: '2.96197679139',
    B11: '4.31725502645',
    B12: '5.92183807954',
    B8: '0.513859419479',
    B9: '3.21637589986',
    B13: '8.98228200978' },
  { index: '2020-04-16 00:10:00',
    B10: '0.705776472345',
    B3: '7.12484267385',
    B4: '1.49799976561',
    B5: '7.13251356121',
    B6: '0.42384778337',
    B7: '0.5732246859',
    B0: '2.86936458419',
    B1: '1.27137235012',
    B2: '2.79434881965',
    B11: '4.16494102534',
    B12: '5.59046554751',
    B8: '0.423153583419',
    B9: '2.9934285503',
    B13: '8.03711889237' } ]

But what we want is a list of `objects`.

         [ {"name":"B10", "values": [
                         {"x":"2020-04-16T06:00:00.000Z", "y":0.857212669594},
                         {"x":"2020-04-16T06:10:00.000Z", "y":0.705776472345},
                         {"x":"2020-04-16T06:20:00.000Z", "y":0.719812464262},... ]}
                         
           {{"name":"B3", "values": [
                         {"x":"2020-04-16T06:00:00.000Z", "y":0.857212669594},
                         {"x":"2020-04-16T06:10:00.000Z", "y":0.705776472345},
                         {"x":"2020-04-16T06:20:00.000Z", "y":0.719812464262},... ]}
           
           ...
          ]
         

### Rearrange

What are the `names`?

In [16]:
var names = d3.keys(load[0]).filter(function(x) { return x !== "index"; })

names.slice(0,5)

[ 'B10', 'B3', 'B4', 'B5', 'B6' ]

Use the `Array.prototype.map` method to create our new array.

In [17]:
var parseDate = d3.time.format("%Y-%m-%d %H:%M:%S").parse;

var load_json = names.map(function(name) {
        return {
          "name": name,
          "values": load.map(function(d) { return { "x": parseDate(d.index), "y": +d[name] }; })
        };
    });

load_json[0]

{ name: 'B10',
  values: 
   [ { x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
       y: 0.857212669594 },
     { x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
       y: 0.705776472345 },
     { x: Thu Apr 16 2020 00:20:00 GMT-0600 (MDT),
       y: 0.719812464262 },
     { x: Thu Apr 16 2020 00:30:00 GMT-0600 (MDT),
       y: 0.705856452329 },
     { x: Thu Apr 16 2020 00:40:00 GMT-0600 (MDT),
       y: 0.709668479812 },
     { x: Thu Apr 16 2020 00:50:00 GMT-0600 (MDT),
       y: 0.706665972836 },
     { x: Thu Apr 16 2020 01:00:00 GMT-0600 (MDT),
       y: 0.704977479627 },
     { x: Thu Apr 16 2020 01:10:00 GMT-0600 (MDT),
       y: 0.700186491876 },
     { x: Thu Apr 16 2020 01:20:00 GMT-0600 (MDT),
       y: 0.706996978359 },
     { x: Thu Apr 16 2020 01:30:00 GMT-0600 (MDT),
       y: 0.696038488204 },
     { x: Thu Apr 16 2020 01:40:00 GMT-0600 (MDT),
       y: 0.702172938959 },
     { x: Thu Apr 16 2020 01:50:00 GMT-0600 (MDT),
       y: 0.696802477428 },
     { x: Thu Apr 16 2020 

####Scales 

In [18]:
var values = _.pluck(load_json, 'values');

values[0]

[ { x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
    y: 0.857212669594 },
  { x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
    y: 0.705776472345 },
  { x: Thu Apr 16 2020 00:20:00 GMT-0600 (MDT),
    y: 0.719812464262 },
  { x: Thu Apr 16 2020 00:30:00 GMT-0600 (MDT),
    y: 0.705856452329 },
  { x: Thu Apr 16 2020 00:40:00 GMT-0600 (MDT),
    y: 0.709668479812 },
  { x: Thu Apr 16 2020 00:50:00 GMT-0600 (MDT),
    y: 0.706665972836 },
  { x: Thu Apr 16 2020 01:00:00 GMT-0600 (MDT),
    y: 0.704977479627 },
  { x: Thu Apr 16 2020 01:10:00 GMT-0600 (MDT),
    y: 0.700186491876 },
  { x: Thu Apr 16 2020 01:20:00 GMT-0600 (MDT),
    y: 0.706996978359 },
  { x: Thu Apr 16 2020 01:30:00 GMT-0600 (MDT),
    y: 0.696038488204 },
  { x: Thu Apr 16 2020 01:40:00 GMT-0600 (MDT),
    y: 0.702172938959 },
  { x: Thu Apr 16 2020 01:50:00 GMT-0600 (MDT),
    y: 0.696802477428 },
  { x: Thu Apr 16 2020 02:00:00 GMT-0600 (MDT),
    y: 0.697226489813 },
  { x: Thu Apr 16 2020 02:10:00 GMT-0600 (MDT),
   

We need to `flatten` or *flatmap* the array or arrays.

In [19]:
values = _.flatten(values)

values.slice(0, 5)

[ { x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
    y: 0.857212669594 },
  { x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
    y: 0.705776472345 },
  { x: Thu Apr 16 2020 00:20:00 GMT-0600 (MDT),
    y: 0.719812464262 },
  { x: Thu Apr 16 2020 00:30:00 GMT-0600 (MDT),
    y: 0.705856452329 },
  { x: Thu Apr 16 2020 00:40:00 GMT-0600 (MDT),
    y: 0.709668479812 } ]

In [20]:
var xvals = _.pluck(values, 'x');
d3.extent(xvals)

[ Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
  Fri Apr 17 2020 00:00:00 GMT-0600 (MDT) ]

In [21]:
var yvals = _.pluck(values, 'y');
d3.extent(yvals)

[ 0.392183399819, 12.2928280401 ]

In [22]:
var x_scale = d3.time.scale()
                .domain(d3.extent(xvals))
                .range([0, 100]);

var y_scale = d3.scale.linear()
                .domain(d3.extent(yvals))
                .range([100, 0]);


undefined

In [23]:
x_scale(parseDate('2020-04-16 00:10:00'))

0.6944444444444444

In [24]:
y_scale(10)

19.266418831962206

###3. Voronoi! 

In [25]:
var nodes;

fs.readFile("data/example_nodes.json", "utf8", function(error, node_data) {
    nodes = JSON.parse(node_data);
});

var x_scale = d3.scale.linear()
                .domain(d3.extent(_.pluck(nodes, "x")))
                .range([0, 100]);

var y_scale = d3.scale.linear()
                .domain(d3.extent(_.pluck(nodes, "y")))
                .range([100, 0]);

nodes.forEach(function(d) {
        d.x = x_scale(d.x);
        d.y = y_scale(d.y); 
      });

nodes.slice(0,5)

[ { y: 12.271046143008101, x: 70.23151622240047, name: 'B11' },
  { y: 89.0652518492427, x: 22.48328225904828, name: 'B0' },
  { y: 67.367470940472, x: 68.6586253932227, name: 'B1' },
  { y: 100, x: 0, name: 'B2' },
  { y: 36.12869848538218, x: 5.148214001870468, name: 'B3' } ]

from [Mike Bostock](https://github.com/mbostock/d3/wiki/Voronoi-Geom)
 
> Returns an array of polygons, one for each input vertex in the specified data array. Each polygon is an array of points, and each point is a two-element array of x and y positions.

In [26]:
var voronoi = d3.geom.voronoi()
        .x(function(d) { return x_scale(d.x); })
        .y(function(d) { return y_scale(d.y); });

voronoi(nodes).slice(0,2)

[ [ [ 31.72431090307535, 281.7238083322705 ],
    [ 42.50781405200926, 364.07505885057714 ],
    [ 286.8084365409115, 140.85599348710764 ],
    [ 132.54135346361488, 143.47878662449483 ],
    point: { y: -169.37, x: 90.19, name: 'B11' } ],
  [ [ -119.08888868070906, -43.43762517942245 ],
    [ 26.287118348384297, -33.977554534615564 ],
    [ 26.78880405622956, -38.068080601929154 ],
    [ -28.527703513187376, -154.3317746511157 ],
    point: { y: 179.46, x: -38.979, name: 'B0' } ] ]

In [27]:
voronoi(nodes).forEach(function(v) { v.point.cell = v; });
nodes

[ { y: -169.37,
    x: 90.19,
    name: 'B11',
    cell: [ [Object], [Object], [Object], [Object], point: [Circular] ] },
  { y: 179.46,
    x: -38.979,
    name: 'B0',
    cell: [ [Object], [Object], [Object], [Object], point: [Circular] ] },
  { y: 80.9,
    x: 85.935,
    name: 'B1',
    cell: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       point: [Circular] ] },
  { y: 229.13,
    x: -99.801,
    name: 'B2',
    cell: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       point: [Circular] ] },
  { y: -60.999,
    x: -85.874,
    name: 'B3',
    cell: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       point: [Circular] ] },
  { y: -225.11,
    x: 141.12,
    name: 'B10',
    cell: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Ob

## 4. Timeseries Voronoi

In [36]:
var load;

fs.readFile("data/example_load.csv", "utf8", function(error, load_data) {
    load = d3.csv.parse(load_data);
});

var names = d3.keys(load[0]).filter(function(x) { return x !== "index"; })
var parseDate = d3.time.format("%Y-%m-%d %H:%M:%S").parse;

var load_json = names.map(function(name) {
        return {
          "name": name,
          "values": load.map(function(d) { return { "x": parseDate(d.index), "y": +d[name] }; })
        };
    });

var values = _.pluck(load_json, 'values');
values = _.flatten(values);

var xvals = _.pluck(values, 'x');
var yvals = _.pluck(values, 'y');

var x_scale = d3.time.scale()
            .domain(d3.extent(xvals))
            .range([0, 100]);

var y_scale = d3.scale.linear()
            .domain(d3.extent(yvals))
            .range([100, 0]);

load_json[0]

{ name: 'B10',
  values: 
   [ { x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
       y: 0.857212669594 },
     { x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
       y: 0.705776472345 },
     { x: Thu Apr 16 2020 00:20:00 GMT-0600 (MDT),
       y: 0.719812464262 },
     { x: Thu Apr 16 2020 00:30:00 GMT-0600 (MDT),
       y: 0.705856452329 },
     { x: Thu Apr 16 2020 00:40:00 GMT-0600 (MDT),
       y: 0.709668479812 },
     { x: Thu Apr 16 2020 00:50:00 GMT-0600 (MDT),
       y: 0.706665972836 },
     { x: Thu Apr 16 2020 01:00:00 GMT-0600 (MDT),
       y: 0.704977479627 },
     { x: Thu Apr 16 2020 01:10:00 GMT-0600 (MDT),
       y: 0.700186491876 },
     { x: Thu Apr 16 2020 01:20:00 GMT-0600 (MDT),
       y: 0.706996978359 },
     { x: Thu Apr 16 2020 01:30:00 GMT-0600 (MDT),
       y: 0.696038488204 },
     { x: Thu Apr 16 2020 01:40:00 GMT-0600 (MDT),
       y: 0.702172938959 },
     { x: Thu Apr 16 2020 01:50:00 GMT-0600 (MDT),
       y: 0.696802477428 },
     { x: Thu Apr 16 2020 

In [37]:
points = [];

load_json.forEach(function(d){
    d.values.forEach(function(v){
        points.push({'name': d.name,
                'x': v.x,
                'y': v.y});
    });
});

points.slice(0, 5)

[ { name: 'B10',
    x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
    y: 0.857212669594 },
  { name: 'B10',
    x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
    y: 0.705776472345 },
  { name: 'B10',
    x: Thu Apr 16 2020 00:20:00 GMT-0600 (MDT),
    y: 0.719812464262 },
  { name: 'B10',
    x: Thu Apr 16 2020 00:30:00 GMT-0600 (MDT),
    y: 0.705856452329 },
  { name: 'B10',
    x: Thu Apr 16 2020 00:40:00 GMT-0600 (MDT),
    y: 0.709668479812 } ]

In [38]:
var voronoi = d3.geom.voronoi()
    .x(function(d) { return x_scale(d.x); })
    .y(function(d) { return y_scale(d.y); });

undefined

In [39]:
voronoi(points).slice(0,2)

[ [ [ -1000000, 96.9932585 ],
    [ -0.13763928782738843, 96.9932585 ],
    [ 0.8930335717128436, 96.43078908279537 ],
    [ 2.0965745836528358, 94.982979159775 ],
    [ 1.3906495351949701, 94.56054078757448 ],
    [ -5.406079440864921, 93.2042905 ],
    [ -1000000, 93.20429049999998 ],
    point: { name: 'B10',
      x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
      y: 0.857212669594 } ],
  [ [ -0.13763928782738843, 96.9932585 ],
    [ 0.5699766727245067, 97.921817 ],
    [ 1.0670461876181778, 97.921817 ],
    [ 1.138866022559751, 97.8782435 ],
    [ 0.8930335717128436, 96.43078908279537 ],
    point: { name: 'B10',
      x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
      y: 0.705776472345 } ] ]

In [32]:
voronoi(points).forEach(function(v) { v.point.cell = v; });
points.slice(0, 2)

[ { name: 'B10',
    x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
    y: 0.857212669594,
    cell: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       point: [Circular] ] },
  { name: 'B10',
    x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
    y: 0.705776472345,
    cell: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       point: [Circular] ] } ]

In [41]:
voronoi(points).map(function(v) { return v }).slice(0,2)

[ [ [ -1000000, 96.9932585 ],
    [ -0.13763928782738843, 96.9932585 ],
    [ 0.8930335717128436, 96.43078908279537 ],
    [ 2.0965745836528358, 94.982979159775 ],
    [ 1.3906495351949701, 94.56054078757448 ],
    [ -5.406079440864921, 93.2042905 ],
    [ -1000000, 93.20429049999998 ],
    point: { name: 'B10',
      x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
      y: 0.857212669594 } ],
  [ [ -0.13763928782738843, 96.9932585 ],
    [ 0.5699766727245067, 97.921817 ],
    [ 1.0670461876181778, 97.921817 ],
    [ 1.138866022559751, 97.8782435 ],
    [ 0.8930335717128436, 96.43078908279537 ],
    point: { name: 'B10',
      x: Thu Apr 16 2020 00:10:00 GMT-0600 (MDT),
      y: 0.705776472345 } ] ]

In [42]:
var res = voronoi(points)

undefined

In [66]:
d = res[0]

[ [ -1000000, 96.9932585 ],
  [ -0.13763928782738843, 96.9932585 ],
  [ 0.8930335717128436, 96.43078908279537 ],
  [ 2.0965745836528358, 94.982979159775 ],
  [ 1.3906495351949701, 94.56054078757448 ],
  [ -5.406079440864921, 93.2042905 ],
  [ -1000000, 93.20429049999998 ],
  point: { name: 'B10',
    x: Thu Apr 16 2020 00:00:00 GMT-0600 (MDT),
    y: 0.857212669594 } ]

In [67]:
d.point.name

'B10'